In [2]:
import requests
from bs4 import BeautifulSoup
import pymongo
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [3]:
import json
f = open("movie.json", "r")
movie_data = json.load(f)
f.close()
movie_data = movie_data[466:699]
print(movie_data[0].keys())
line = ""
for i in movie_data[0:100]:
    line += "{}\t".format(i['Mname'])
print(line)

[u'realease_kor', u'Webpage', u'Mname', u'Domestic', u'budget', u'gross_kor', u'release_usa', u'year', u'runtime', u'Oversea', u'rating_usa']
Pieta	Lovelace	Disconnect	Adore	Chronic	Transformers: Age of Extinction	The Hobbit: The Battle of the Five Armies	Guardians of the Galaxy	Maleficent	The Hunger Games: Mockingjay - Part 1	X-Men: Days of Future Past	Captain America: The Winter Soldier	Dawn of the Planet of the Apes	The Amazing Spider-Man 2	Interstellar	Big Hero 6	How to Train Your Dragon 2	American Sniper	Godzilla (2014)	Rio 2	Teenage Mutant Ninja Turtles (2014)	The LEGO Movie	Lucy	Penguins of Madagascar	Edge of Tomorrow	Gone Girl	Night at the Museum: Secret of the Tomb	Noah	The Maze Runner	300: Rise of An Empire	The Fault in our Stars	Divergent	Mr. Peabody & Sherman	Neighbors	Exodus: Gods and Kings	Annabelle	Hercules (2014)	RoboCop (2014)	The Imitation Game	Non-Stop	Dracula Untold	The Expendables 3	Into the Woods	Fury (2014)	Need for Speed	The Other Woman (2014)	The Equalizer	The 

In [88]:
class Database(object):
    def __init__(self):
        self.connection = pymongo.MongoClient("localhost", 27017)
        self.db = self.connection["DSS4_SmallP"]
        self.collection = self.db["Movie_mojo"]        
def Load_Data_mojo_yearly_dom(page,year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?page={}&view=releasedate&view2=domestic&yr={}&p=.htm'.format(page,year)
    return requests.get(Mojo_url)
def Load_Data_mojo_yearly_intl(year):
    Mojo_url = u'http://www.boxofficemojo.com/'\
        u'yearly/chart/?view2=worldwide&yr={}&p=.htm'.format(year)
    return requests.get(Mojo_url)
def Load_Data_mojo_movie_intl(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}&page=intl'.format(M_id)
    return requests.get(Mojo_url)
def Load_Data_mojo_movie(M_id):
    Mojo_url = u'http://www.boxofficemojo.com{}'.format(M_id)
    return requests.get(Mojo_url)
def load_apikeys():
    fp = open("private.json","r")
    res = json.load(fp)
    return res
def load_data_daum(query):
    key = load_apikeys()['daum_key']
    url = "https://apis.daum.net/contents/movie?apikey={}&q={}&output=json".format(key, query)
#    print(url)
    return requests.get(url)


In [89]:
print(movie_data[0].keys())

[u'realease_kor', u'Webpage', u'Mname', u'Domestic', u'budget', u'gross_kor', u'release_usa', u'year', u'runtime', u'Oversea', u'rating_usa']


In [92]:
import re
def select_daum(query, movie):
    res = load_data_daum(query)
#    print(res)
    data = json.loads(res.content)
    if data['channel']['totalCount'] == 0:
        query2 = re.match('[a-zA-Z\s]+',query)
        if query2.group() == query:
            return query
        else:
            return select_daum(query2.group(), movie)
    elif data['channel']['totalCount'] == 1:
        return data['channel']["item"][0]
    else :
        mlist = []
        m = movie['realease_kor'].split('/')
        m = '{0:02d}{1:02d}{2:02d}'.format(int(m[2]),int(m[0]),int(m[1]))
        for item in data['channel']['item']:
            try:
                d = item['open_info'][0]['content'].split('.')
                d = '{0:02d}{1:02d}{2:02d}'.format(int(d[0][-2:]),int(d[1]),int(d[2]))
                if d == m :
                    mlist.append(item)
            except:
                pass
        if len(mlist) == 1:
            return mlist[0]
        else :
            if len(mlist) != 0 :print(len(mlist))
            return -1
        
        
for movie in movie_data:
    res = select_daum(movie['Mname'],movie)
    if res == -1 :
        print("re : {}".format(movie['Mname']))
#        break
    elif type(res) != dict : 
        print("none : {}".format(res))

none : Penguins of Madagascar
re : Neighbors
re : Hercules (2014)
none : The Admiral
none : Thunder and the House of Magic
none : Frank Miller
none : Like Father
none : Legends of Oz
none : Nymphomaniac
none : The Disappearance of Eleanor Rigby
re : The Signal (2014)
none : Furious 
none : The Peanuts Movie
re : Everest (2015)
re : The Intern
re : Focus (2015)
none : Dragon Ball Z
re : The Walk
re : Love & Mercy
none : Baahubali


In [87]:
print(movie)

{u'realease_kor': u'7/3/14', u'Webpage': u'/movies/?id=townies.htm', u'Mname': u'Neighbors', u'Domestic': u'$150.2', u'budget': u'$18 million', u'gross_kor': 215659.0, u'release_usa': u'May 9, 2014', u'year': 2014, u'runtime': 96.0, u'Oversea': u'$120.5', u'rating_usa': u'R'}


In [84]:
res = load_data_daum('Neighbors')
data = json.loads(res.content)
print(data)

https://apis.daum.net/contents/movie?apikey=ca6800712ed274862ce0793b72985180&q=Neighbors&output=json
{u'channel': {u'item': [{u'audience_date': [{u'content': u'', u'link': u''}], u'photo4': {u'content': u'http://t1.search.daumcdn.net/thumb/R438x0.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fmovie%2Fbfde1b7b51f5967a2ae43f5156e7f02668054544', u'link': u'http://movie.daum.net/moviedb/photoviewer?id=94284#1027268'}, u'photo5': {u'content': u'', u'link': u''}, u'photo2': {u'content': u'http://t1.search.daumcdn.net/thumb/R438x0.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fmovie%2F2a839617e2ed0b014d5498dadf81573f37a5113c', u'link': u'http://movie.daum.net/moviedb/photoviewer?id=94284#1027265'}, u'photo3': {u'content': u'http://t1.search.daumcdn.net/thumb/R438x0.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fmovie%2F8adfef175861d09c3b13ccf6dff0367a51e5f9fa', u'link': u'http://movie.daum.net/moviedb/photoviewer?id=94284#1027267'}, u'grades': [{u'content': u'', u'link': u'http://movie.daum.net/moviedb/grade?m

In [39]:
print(res.content)

{"channel":{"result":10,"title":"Search SCC Open API","page":1,"totalCount":4,"q":"lucy","item":[{"genre":[{"content":"액션","link":""},{"content":"SF","link":""},{"content":"스릴러","link":""}],"res":[{"content":"","link":""}],"trailer":[{"content":"http://movie.daum-img.net/movie/movie-trailer/35/53/45335/trailer_45335.jpg","link":"http://movie.daum.net/moviedb/video?id=81452&vclipId=45335"}],"photo1":{"content":"http://t1.search.daumcdn.net/thumb/R438x0.q85/?fname=http%3A%2F%2Fcfile78.uf.daum.net%2Fimage%2F222D934F53E25A7A1CFA5F","link":"http://movie.daum.net/moviedb/photoviewer?id=81452#946162"},"actor":[{"content":"스칼렛 요한슨","link":"http://movie.daum.net/person/main?personId=11739"},{"content":"모건 프리먼","link":"http://movie.daum.net/person/main?personId=828"},{"content":"최민식","link":"http://movie.daum.net/person/main?personId=562"},{"content":"아므르 웨이크드","link":"http://movie.daum.net/person/main?personId=238317"},{"content":"줄리안 린드-터트","link":"http://movie.daum.net/person/main?personId=17

In [ ]:
for name in dbs:
    tmp = dbs[name]
    M_id = tmp['Webpage']
    res = Load_Data_mojo_movie_intl(M_id)
    content = res.content
    soup = BeautifulSoup(content,'html.parser')
    
    table = soup.find('table', attrs = { "style" : "padding-top: 5px;"})
    tds = table.find_all('td')
    #<table border="0" cellspacing="1" cellpadding="3">
    table2 = soup.find('table', attrs = { "border" : "0", "cellspacing" : "1", "cellpadding" : "3"})
    tds2 = table2.find_all('td')
    
    for n, feature in enumerate(tds2):
        if feature.get_text() == 'South Korea':
            dbs[name].update({
                'rating_usa' : tds[8].b.get_text(),
                'release_usa' : tds[5].b.get_text(),
                'realease_kor' : tds2[n+2].get_text(),
                'gross_kor' : tds2[n+5].get_text(),
                'budget' : tds[9].b.get_text()
            })
            break
    else:
        try:
            budget = tds[9].b.get_text()
        except:
            budget = 'N/A'
        dbs[name].update({
            'rating_usa' : tds[8].b.get_text(),
            'release_usa' : tds[5].b.get_text(),
            'budget' : budget
        })
    time.sleep(1)
    print dbs[name]
    i += 1
    if i == 10 :break

In [3]:
#mojo -> 시리즈, 제작비, 감독, 배우
for movie in movie_data:
    a


233
